In [1]:
import keras
import tensorflow as tf
import librosa
import numpy as np
import scipy
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:
model_path = '../models/model_2/CNN_model.keras'
model = keras.models.load_model(model_path)
model.summary()

c:\Users\devar\miniconda3\envs\streamlit\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 25 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_28 (Conv1D)              │ (None, 154, 512)       │         3,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_28 (MaxPooling1D) │ (None, 77, 512)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_29 (Conv1D)              │ (None, 77, 256)        │       655,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_29 (MaxPooling1D) │ (None, 39, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_30 (Conv1D)              │ (None, 39, 128)        │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_30 (MaxPooling1D) │ (None, 20, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 20, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_31 (Conv1D)              │ (None, 20, 64)         │        41,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_31 (MaxPooling1D) │ (None, 10, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 640)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,839,895 (10.83 MB)

 Trainable params: 946,631 (3.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,893,264 (7.22 MB)

In [3]:
# Function to extract features
def extract_features(data_path):
    
    data, sample_rate = librosa.load(data_path)

    # MFCCs
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)

    # Energy
    energy = np.mean(librosa.feature.rms(y=data).T, axis=0)

    # Entropy of energy (computed as entropy of the short-term energy)
    st_energy = librosa.feature.rms(y=data, frame_length=2048, hop_length=512)
    entropy_of_energy = np.mean([scipy.stats.entropy(e) for e in st_energy])

    # Zero Crossing Rate
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)

    # Mel-Spectrogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)

    # Spectral Centroid
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=data, sr=sample_rate).T, axis=0)

    # Spectral Spread
    spectral_spread = np.mean(librosa.feature.spectral_bandwidth(y=data, sr=sample_rate).T, axis=0)

    # Spectral Roll-off
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=data, sr=sample_rate).T, axis=0)

    # Concatenate all features
    features = np.hstack((mfcc, energy, entropy_of_energy, zcr, mel, spectral_centroid, spectral_spread, spectral_rolloff))
    return features

data_path = "../data/input/Crema/1001_ITH_HAP_XX.wav" 
X = tf.expand_dims(extract_features(data_path), axis=-1)


In [4]:
X

<tf.Tensor: shape=(154, 1), dtype=float64, numpy=
array([[-3.33686249e+02],
       [ 1.34182343e+02],
       [-1.76158257e+01],
       [ 3.33083267e+01],
       [-3.09888911e+00],
       [ 1.72047558e+01],
       [-2.16867905e+01],
       [-1.60438418e+00],
       [-1.15475712e+01],
       [-3.25284386e+00],
       [-8.61373138e+00],
       [-1.05538547e+00],
       [-3.13319659e+00],
       [-1.62905235e+01],
       [ 4.48697329e+00],
       [-1.32159090e+01],
       [-3.66729069e+00],
       [-8.12562656e+00],
       [-3.49940681e+00],
       [-6.82263470e+00],
       [ 3.69887985e-02],
       [ 4.09885053e+00],
       [ 6.44211814e-02],
       [ 2.10046535e-03],
       [ 8.35992210e-03],
       [ 3.35056745e-02],
       [ 2.60696709e-01],
       [ 1.14967811e+00],
       [ 1.84425807e+00],
       [ 9.27966177e-01],
       [ 1.76566517e+00],
       [ 1.17489171e+00],
       [ 8.97790074e-01],
       [ 1.35326648e+00],
       [ 1.77778673e+00],
       [ 9.87669349e-01],
       [ 1.121

In [5]:
# Scaling the data using StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
# X = tf.squeeze(X)
X

array([[-1.35669248e+00],
       [ 3.63309716e-01],
       [-1.94738392e-01],
       [-7.52844585e-03],
       [-1.41370495e-01],
       [-6.67292082e-02],
       [-2.09704279e-01],
       [-1.35876320e-01],
       [-1.72429968e-01],
       [-1.41936471e-01],
       [-1.61644437e-01],
       [-1.33858064e-01],
       [-1.41496618e-01],
       [-1.89866249e-01],
       [-1.13482964e-01],
       [-1.78563196e-01],
       [-1.43460082e-01],
       [-1.59850042e-01],
       [-1.42842898e-01],
       [-1.55059917e-01],
       [-1.29842222e-01],
       [-1.14909800e-01],
       [-1.29741374e-01],
       [-1.29970480e-01],
       [-1.29947469e-01],
       [-1.29855027e-01],
       [-1.29019816e-01],
       [-1.25751697e-01],
       [-1.23198247e-01],
       [-1.26566766e-01],
       [-1.23487174e-01],
       [-1.25659006e-01],
       [-1.26677701e-01],
       [-1.25003255e-01],
       [-1.23442613e-01],
       [-1.26347282e-01],
       [-1.25856834e-01],
       [-1.22560477e-01],
       [-1.2

In [11]:
# Reshape 
X = tf.expand_dims(X, axis=0)
X.shape

TensorShape([1, 154, 1])

In [7]:
# List of classes
classes = ['angry', 'calm', 'disgust', 'fear', 'happy', 'sad', 'surprise']

In [12]:
pred = model.predict(X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step


In [13]:
pred

array([[0.12231044, 0.00413417, 0.10876969, 0.7255662 , 0.01023937,
        0.0020535 , 0.02692665]], dtype=float32)